In [ ]:
"""
Dashboard Interactivo para Simulación de Cruce Peatonal Inteligente
Ejecutar en Jupyter Notebook o JupyterLab
"""

# ============================================================================
# CELDA 1: Imports y configuración
# ============================================================================
import sys
import os

# Agregar directorio padre al path
sys.path.insert(0, os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Importar módulos del proyecto
from config import *
from test_semaforo_fijo import simular_semaforo_fijo
from test_semaforo_adaptativo import simular_semaforo_adaptativo
from analisis.metricas import AnalizadorMetricas, imprimir_resumen_metricas
from analisis.visualizaciones import VisualizadorSimulacion

# Configurar estilo
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 6)

print("✅ Módulos importados correctamente")
print("📊 Dashboard listo para usar")

# ============================================================================
# CELDA 2: Función principal de simulación interactiva
# ============================================================================

def simular_interactivo(tipo_control='Adaptativo',
                       lambda_v=0.3,
                       lambda_p=0.1,
                       g_min=20,
                       g_max=60,
                       g_v_fijo=30,
                       g_p_fijo=15,
                       t_v=5,
                       t_p=3,
                       b_extension=5,
                       w_max=90,
                       tiempo_sim=1800,
                       mostrar_graficos=True):
    """
    Función interactiva para ejecutar simulaciones con parámetros personalizados.
    """
    
    clear_output(wait=True)
    
    print("="*70)
    print(f"🚀 EJECUTANDO SIMULACIÓN: {tipo_control.upper()}")
    print("="*70)
    print(f"\nParámetros:")
    print(f"  λ_v = {lambda_v} veh/seg ({lambda_v*3600:.0f} veh/hora)")
    print(f"  λ_p = {lambda_p} peat/seg ({lambda_p*3600:.0f} peat/hora)")
    
    if tipo_control == 'Fijo':
        print(f"  Verde vehicular = {g_v_fijo}s")
        print(f"  Verde peatonal  = {g_p_fijo}s")
    else:
        print(f"  Verde vehicular = [{g_min}, {g_max}]s")
        print(f"  Umbrales: T_v={t_v}, T_p={t_p}, b={b_extension}s, W_max={w_max}s")
    
    print(f"  Tiempo simulación = {tiempo_sim}s")
    print("="*70 + "\n")
    
    # Ejecutar simulación
    if tipo_control == 'Fijo':
        registros, _, _ = simular_semaforo_fijo(
            tiempo_sim=tiempo_sim,
            lambda_v=lambda_v,
            lambda_p=lambda_p,
            g_v=g_v_fijo,
            g_p=g_p_fijo,
            warmup=min(300, tiempo_sim//10),
            semilla=42,
            verbose=False
        )
    else:  # Adaptativo
        registros, _, _ = simular_semaforo_adaptativo(
            tiempo_sim=tiempo_sim,
            lambda_v=lambda_v,
            lambda_p=lambda_p,
            g_min=g_min,
            g_max=g_max,
            g_p=g_p_fijo,
            t_v=t_v,
            t_p=t_p,
            b=b_extension,
            w_max=w_max,
            warmup=min(300, tiempo_sim//10),
            semilla=42,
            verbose=False
        )
    
    # Analizar resultados
    warmup = min(300, tiempo_sim//10)
    analizador = AnalizadorMetricas(registros, tiempo_sim, warmup)
    resumen = analizador.generar_resumen_completo()
    
    # Imprimir métricas
    imprimir_resumen_metricas(resumen)
    
    # Mostrar gráficos si se solicita
    if mostrar_graficos:
        print("\n📊 Generando visualizaciones...\n")
        
        df_estado = analizador.crear_df_estado_colas()
        df_v = analizador.crear_df_servicios_vehiculos()
        df_p = analizador.crear_df_servicios_peatones()
        
        # Crear figura con 3 subplots
        fig, axes = plt.subplots(3, 1, figsize=(14, 10))
        
        # Subplot 1: Series temporales de colas
        if not df_estado.empty:
            axes[0].plot(df_estado['tiempo'], df_estado['cola_v'], 
                        label='Vehículos', color='#2E86AB', linewidth=1.5)
            axes[0].plot(df_estado['tiempo'], df_estado['cola_p'],
                        label='Peatones', color='#A23B72', linewidth=1.5)
            axes[0].set_ylabel('Longitud de cola', fontweight='bold')
            axes[0].set_title('Evolución de Colas', fontweight='bold', fontsize=12)
            axes[0].legend()
            axes[0].grid(True, alpha=0.3)
        
        # Subplot 2: Histograma de espera vehicular
        if not df_v.empty:
            axes[1].hist(df_v['tiempo_espera'], bins=30, color='#2E86AB', 
                        alpha=0.7, edgecolor='black')
            axes[1].axvline(df_v['tiempo_espera'].mean(), color='red', 
                           linestyle='--', linewidth=2, 
                           label=f"Media: {df_v['tiempo_espera'].mean():.1f}s")
            axes[1].set_xlabel('Tiempo de espera (s)', fontweight='bold')
            axes[1].set_ylabel('Frecuencia', fontweight='bold')
            axes[1].set_title('Distribución de Espera - Vehículos', 
                             fontweight='bold', fontsize=12)
            axes[1].legend()
            axes[1].grid(True, alpha=0.3)
        
        # Subplot 3: Histograma de espera peatonal
        if not df_p.empty:
            axes[2].hist(df_p['tiempo_espera'], bins=30, color='#A23B72',
                        alpha=0.7, edgecolor='black')
            axes[2].axvline(df_p['tiempo_espera'].mean(), color='red',
                           linestyle='--', linewidth=2,
                           label=f"Media: {df_p['tiempo_espera'].mean():.1f}s")
            axes[2].set_xlabel('Tiempo de espera (s)', fontweight='bold')
            axes[2].set_ylabel('Frecuencia', fontweight='bold')
            axes[2].set_title('Distribución de Espera - Peatones',
                             fontweight='bold', fontsize=12)
            axes[2].legend()
            axes[2].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
    return resumen, analizador

# ============================================================================
# CELDA 3: Widgets del dashboard
# ============================================================================

# Crear widgets
tipo_control_widget = widgets.Dropdown(
    options=['Adaptativo', 'Fijo'],
    value='Adaptativo',
    description='Control:',
    style={'description_width': '120px'}
)

lambda_v_widget = widgets.FloatSlider(
    value=0.3,
    min=0.1,
    max=0.6,
    step=0.05,
    description='λ_v (veh/s):',
    style={'description_width': '120px'},
    continuous_update=False
)

lambda_p_widget = widgets.FloatSlider(
    value=0.1,
    min=0.05,
    max=0.3,
    step=0.05,
    description='λ_p (peat/s):',
    style={'description_width': '120px'},
    continuous_update=False
)

# Parámetros control fijo
g_v_fijo_widget = widgets.IntSlider(
    value=30,
    min=15,
    max=60,
    step=5,
    description='Verde veh (s):',
    style={'description_width': '120px'},
    continuous_update=False
)

g_p_fijo_widget = widgets.IntSlider(
    value=15,
    min=10,
    max=30,
    step=5,
    description='Verde peat (s):',
    style={'description_width': '120px'},
    continuous_update=False
)

# Parámetros control adaptativo
g_min_widget = widgets.IntSlider(
    value=20,
    min=10,
    max=30,
    step=5,
    description='G_min (s):',
    style={'description_width': '120px'},
    continuous_update=False
)

g_max_widget = widgets.IntSlider(
    value=60,
    min=40,
    max=90,
    step=10,
    description='G_max (s):',
    style={'description_width': '120px'},
    continuous_update=False
)

t_v_widget = widgets.IntSlider(
    value=5,
    min=2,
    max=10,
    step=1,
    description='T_v (veh):',
    style={'description_width': '120px'},
    continuous_update=False
)

t_p_widget = widgets.IntSlider(
    value=3,
    min=1,
    max=6,
    step=1,
    description='T_p (peat):',
    style={'description_width': '120px'},
    continuous_update=False
)

b_extension_widget = widgets.IntSlider(
    value=5,
    min=3,
    max=10,
    step=1,
    description='b (s):',
    style={'description_width': '120px'},
    continuous_update=False
)

w_max_widget = widgets.IntSlider(
    value=90,
    min=60,
    max=150,
    step=10,
    description='W_max (s):',
    style={'description_width': '120px'},
    continuous_update=False
)

tiempo_sim_widget = widgets.IntSlider(
    value=1800,
    min=600,
    max=3600,
    step=300,
    description='Tiempo sim (s):',
    style={'description_width': '120px'},
    continuous_update=False
)

mostrar_graficos_widget = widgets.Checkbox(
    value=True,
    description='Mostrar gráficos',
    style={'description_width': '120px'}
)

# Botón de ejecución
ejecutar_button = widgets.Button(
    description='🚀 EJECUTAR SIMULACIÓN',
    button_style='success',
    layout=widgets.Layout(width='300px', height='50px')
)

# ============================================================================
# CELDA 4: Layout del dashboard
# ============================================================================

# Sección de parámetros generales
seccion_general = widgets.VBox([
    widgets.HTML("<h3>⚙️ Parámetros Generales</h3>"),
    tipo_control_widget,
    lambda_v_widget,
    lambda_p_widget,
    tiempo_sim_widget,
    mostrar_graficos_widget
])

# Sección de parámetros de control fijo
seccion_fijo = widgets.VBox([
    widgets.HTML("<h3>🚦 Control Fijo</h3>"),
    g_v_fijo_widget,
    g_p_fijo_widget
])

# Sección de parámetros de control adaptativo
seccion_adaptativo = widgets.VBox([
    widgets.HTML("<h3>🎛️ Control Adaptativo</h3>"),
    g_min_widget,
    g_max_widget,
    t_v_widget,
    t_p_widget,
    b_extension_widget,
    w_max_widget
])

# Layout principal
layout_principal = widgets.HBox([
    seccion_general,
    seccion_fijo,
    seccion_adaptativo
])

# Botón centrado
boton_layout = widgets.HBox([ejecutar_button], 
                            layout=widgets.Layout(justify_content='center'))

# Dashboard completo
dashboard = widgets.VBox([
    widgets.HTML("<h1 style='text-align: center;'>🚦 Dashboard de Simulación de Cruce Peatonal Inteligente</h1>"),
    widgets.HTML("<hr>"),
    layout_principal,
    widgets.HTML("<hr>"),
    boton_layout
])

# Output area para resultados
output_area = widgets.Output()

def on_button_clicked(b):
    """Callback para el botón de ejecución."""
    with output_area:
        simular_interactivo(
            tipo_control=tipo_control_widget.value,
            lambda_v=lambda_v_widget.value,
            lambda_p=lambda_p_widget.value,
            g_min=g_min_widget.value,
            g_max=g_max_widget.value,
            g_v_fijo=g_v_fijo_widget.value,
            g_p_fijo=g_p_fijo_widget.value,
            t_v=t_v_widget.value,
            t_p=t_p_widget.value,
            b_extension=b_extension_widget.value,
            w_max=w_max_widget.value,
            tiempo_sim=tiempo_sim_widget.value,
            mostrar_graficos=mostrar_graficos_widget.value
        )

ejecutar_button.on_click(on_button_clicked)

# Mostrar dashboard
display(dashboard)
display(output_area)

# ============================================================================
# CELDA 5: Comparación rápida Fijo vs Adaptativo
# ============================================================================

def comparacion_rapida(lambda_v=0.3, lambda_p=0.1, tiempo_sim=1800):
    """
    Compara rápidamente ambos controladores con los parámetros dados.
    """
    print("="*70)
    print("⚡ COMPARACIÓN RÁPIDA: FIJO vs ADAPTATIVO")
    print("="*70 + "\n")
    
    # Simular ambos
    print("[1/2] Simulando FIJO...")
    registros_fijo, _, _ = simular_semaforo_fijo(
        tiempo_sim=tiempo_sim,
        lambda_v=lambda_v,
        lambda_p=lambda_p,
        warmup=min(300, tiempo_sim//10),
        semilla=42,
        verbose=False
    )
    
    print("[2/2] Simulando ADAPTATIVO...")
    registros_adapt, _, _ = simular_semaforo_adaptativo(
        tiempo_sim=tiempo_sim,
        lambda_v=lambda_v,
        lambda_p=lambda_p,
        warmup=min(300, tiempo_sim//10),
        semilla=42,
        verbose=False
    )
    
    # Analizar
    warmup = min(300, tiempo_sim//10)
    analizador_fijo = AnalizadorMetricas(registros_fijo, tiempo_sim, warmup)
    analizador_adapt = AnalizadorMetricas(registros_adapt, tiempo_sim, warmup)
    
    resumen_fijo = analizador_fijo.generar_resumen_completo()
    resumen_adapt = analizador_adapt.generar_resumen_completo()
    
    # Tabla comparativa
    print("\n" + "="*70)
    print(f"{'Métrica':<35} {'FIJO':>15} {'ADAPTATIVO':>15}")
    print("-"*70)
    
    if resumen_fijo.get('metricas_espera_vehiculos') and resumen_adapt.get('metricas_espera_vehiculos'):
        w_v_f = resumen_fijo['metricas_espera_vehiculos']['espera_media']
        w_v_a = resumen_adapt['metricas_espera_vehiculos']['espera_media']
        print(f"{'Espera media vehículos (s)':<35} {w_v_f:>15.2f} {w_v_a:>15.2f}")
        
        p95_v_f = resumen_fijo['metricas_espera_vehiculos']['percentil_95']
        p95_v_a = resumen_adapt['metricas_espera_vehiculos']['percentil_95']
        print(f"{'Percentil 95 vehículos (s)':<35} {p95_v_f:>15.2f} {p95_v_a:>15.2f}")
    
    if resumen_fijo.get('metricas_espera_peatones') and resumen_adapt.get('metricas_espera_peatones'):
        w_p_f = resumen_fijo['metricas_espera_peatones']['espera_media']
        w_p_a = resumen_adapt['metricas_espera_peatones']['espera_media']
        print(f"{'Espera media peatones (s)':<35} {w_p_f:>15.2f} {w_p_a:>15.2f}")
        
        p95_p_f = resumen_fijo['metricas_espera_peatones']['percentil_95']
        p95_p_a = resumen_adapt['metricas_espera_peatones']['percentil_95']
        print(f"{'Percentil 95 peatones (s)':<35} {p95_p_f:>15.2f} {p95_p_a:>15.2f}")
    
    print("="*70 + "\n")
    
    # Gráfico comparativo
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Comparación espera vehicular
    df_v_f = analizador_fijo.crear_df_servicios_vehiculos()
    df_v_a = analizador_adapt.crear_df_servicios_vehiculos()
    
    if not df_v_f.empty and not df_v_a.empty:
        axes[0].hist(df_v_f['tiempo_espera'], bins=20, alpha=0.6, 
                    label='Fijo', color='#2E86AB', edgecolor='black')
        axes[0].hist(df_v_a['tiempo_espera'], bins=20, alpha=0.6,
                    label='Adaptativo', color='#28A745', edgecolor='black')
        axes[0].set_xlabel('Tiempo de espera (s)', fontweight='bold')
        axes[0].set_ylabel('Frecuencia', fontweight='bold')
        axes[0].set_title('Vehículos', fontweight='bold', fontsize=13)
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
    
    # Comparación espera peatonal
    df_p_f = analizador_fijo.crear_df_servicios_peatones()
    df_p_a = analizador_adapt.crear_df_servicios_peatones()
    
    if not df_p_f.empty and not df_p_a.empty:
        axes[1].hist(df_p_f['tiempo_espera'], bins=20, alpha=0.6,
                    label='Fijo', color='#A23B72', edgecolor='black')
        axes[1].hist(df_p_a['tiempo_espera'], bins=20, alpha=0.6,
                    label='Adaptativo', color='#FF6B9D', edgecolor='black')
        axes[1].set_xlabel('Tiempo de espera (s)', fontweight='bold')
        axes[1].set_ylabel('Frecuencia', fontweight='bold')
        axes[1].set_title('Peatones', fontweight='bold', fontsize=13)
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)
    
    plt.suptitle('Comparación: Fijo vs Adaptativo', fontsize=15, fontweight='bold')
    plt.tight_layout()
    plt.show()

# Widget para comparación rápida
comparacion_widget = interactive(
    comparacion_rapida,
    lambda_v=widgets.FloatSlider(value=0.3, min=0.1, max=0.6, step=0.05, description='λ_v:'),
    lambda_p=widgets.FloatSlider(value=0.1, min=0.05, max=0.3, step=0.05, description='λ_p:'),
    tiempo_sim=widgets.IntSlider(value=1800, min=600, max=3600, step=300, description='Tiempo (s):')
)

print("\n📊 Para ejecutar comparación rápida, ejecuta:")
print("display(comparacion_widget)")